In [ ]:
#=============================================================================================#
#Name: mimic_db_and_rxclass.pynb
#Programmer: Inna Cohen
#Date Modified: December 12, 2022
#Purpose: Thise code creates the mimic.db file and utilizes the RxClass API
#=============================================================================================#

In [ ]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import requests
from IPython.display import display

In [ ]:

def get_drug_name(classId, relaSource):
    my_url = 'https://rxnav.nlm.nih.gov/REST/rxclass/classMembers.json?classId=' + classId + '&relaSource=' + relaSource
    r = requests.get(url = my_url)
    data = r.json()
    terms = data['drugMemberGroup']['drugMember']
    lst = []

    for n, term in enumerate(terms):
        rxcui = terms[n]['minConcept']['name']
        lst.append(rxcui)
    
    return lst



def View(df, rows=False):
    '''Views all the columns of a dataframe '''
    if rows==True:
        with pd.option_context("display.max_columns", None, "display.max_rows", None):
            display(df)
    else:
        with pd.option_context("display.max_columns", None):
            display(df)

In [ ]:
opiods = get_drug_name('N02A','ATC')
benzos = get_drug_name('D001569','DAILYMED')

In [ ]:
conn = sqlite3.connect('mimic.db')
cursor = conn.cursor()

### Creating tables

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Patients")
cursor.execute("DROP TABLE IF EXISTS Admissions")
cursor.execute("DROP TABLE IF EXISTS Diagnoses_icd")
cursor.execute("DROP TABLE IF EXISTS D_icd_diagnoses")
cursor.execute("DROP TABLE IF EXISTS Prescriptions")
cursor.execute("DROP TABLE IF EXISTS Drugs")
cursor.execute("DROP TABLE IF EXISTS Labevents")



sql ='''CREATE TABLE Patients (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    gender varchar(5)  NOT NULL,
    dob datetime  NOT NULL,
    dod datetime  NOT NULL,
    dod_hosp datetime  NULL,
    dod_ssn datetime  NULL,
    expire_flag varchar(5)  NOT NULL,
    PRIMARY KEY (subject_id)
  );'''

cursor.execute(sql)


sql ='''CREATE TABLE Admissions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    admittime datetime  NOT NULL,
    dischtime datetime  NOT NULL,
    deathtime datetime   NULL,
    admission_type varchar(50)  NOT NULL,
    admission_location varchar(50)  NOT NULL,
    discharge_location varchar(50)  NOT NULL,
    insurance varchar(255)  NOT NULL,
    language varchar(10)   NULL,
    religion varchar(50)   NULL,
    marital_status varchar(50)   NULL,
    ethnicity varchar(200)  NOT NULL,
    edregtime datetime   NULL,
    edouttime datetime   NULL,
    diagnosis varchar(300)  NOT NULL,
    hospital_expire_flag int  NOT NULL,
    has_chartevents_data int  NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''

cursor.execute(sql)

sql ='''CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10) NOT NULL,
    short_title varchar(50) NOT NULL,
    long_title varchar(300) NOT NULL,
    PRIMARY KEY (icd9_code) 
  );'''
cursor.execute(sql)


sql ='''CREATE TABLE Diagnoses_icd (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
     seq_num int NOT NULL,
     icd9_code varchar(10) NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(icd9_code)
  );'''
cursor.execute(sql)


 #insert code to create Prescriptions table
sql ='''CREATE TABLE Prescriptions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    startdate datetime NULL,
    enddate datetime NULL,
    drug_type varchar(50) NULL,
    drug varchar(300) NULL,
    drug_name_poe varchar(300) NULL,
    drug_name_generic varchar(300) NULL,
    formulary_drug_cd varchar(300) NULL,
    gsn int NULL,
    ndc real NULL,
    prod_strength varchar(300) NULL,
    dose_val_rx varchar(50) NULL,
    dose_unit_rx varchar(50) NULL,
    form_val_disp varchar(50) NULL,
    form_unit_disp varchar(50) NULL,
    route varchar(50) NULL,
    FOREIGN KEY (hadm_id) REFERENCES Diagnoses_icd(hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''
cursor.execute(sql)

sql ='''CREATE TABLE Labevents (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    itemid NULL,
    chartime datetime NULL,
    value varchar(200) NULL, 
    valuenum float8 NULL,
    valueuom varchar(20) NULL,
    flag varchar(200) NULL,
    FOREIGN KEY (hadm_id) REFERENCES Diagnoses_icd(hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''
cursor.execute(sql)


print("Table(s) created successfully........")
conn.commit()

### Data Cleaning 

In [ ]:
patients = pd.read_csv('../../mimic-iii-clinical-database-1.4/PATIENTS.csv.gz', compression='gzip').dropna()

In [ ]:
admissions = pd.read_csv('../../mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz', compression='gzip').dropna()

In [ ]:
d_icd_diagnoses = pd.read_csv('../../mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz', compression='gzip').dropna()

In [ ]:
diagnoses_icd = pd.read_csv('../../mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz', compression='gzip').dropna()

In [ ]:
prescriptions = pd.read_csv('../../mimic-iii-clinical-database-1.4/PRESCRIPTIONS.csv.gz', compression='gzip', low_memory=False).dropna()

In [ ]:
labevents = pd.read_csv('../../mimic-iii-clinical-database-1.4/LABEVENTS.csv.gz', compression='gzip', low_memory=False).dropna()

In [ ]:
def find_drug(drug):
    return prescriptions.DRUG[prescriptions.DRUG.str.contains(drug, case=False, regex=True)].value_counts().to_frame()

In [ ]:
def clean_drug(drug):
    return prescriptions.DRUG[prescriptions.DRUG.str.replace(drug, drug, case=False, regex=True)]

In [ ]:
opiod_data = []
for opiod in opiods:
    data = find_drug(opiod)
    data['name'] = opiod
    data['class'] = 'opiods'
    opiod_data.append(data)
opiod_data = pd.concat(opiod_data)

In [ ]:
benzo_data = []
for benzo in benzos:
    data = find_drug(benzo)
    data['name'] = benzo
    data['class'] = 'benzodiazepines'
    benzo_data.append(data)
benzo_data = pd.concat(benzo_data)

In [ ]:
opiod_data.reset_index(inplace=True)
benzo_data.reset_index(inplace=True)

In [ ]:
drug_data = pd.concat([opiod_data, benzo_data], ignore_index=True)

In [ ]:
drug_data = drug_data.drop(["DRUG"],axis=1)

In [ ]:
drug_data.columns = ['drug_id','name','class']

In [ ]:
drug_data.to_csv("drug_data.csv", index=False)

In [ ]:
drug_data = pd.read_csv("drug_data.csv")

In [ ]:
sql ='''CREATE TABLE Drugs (
    drug_id varchar(300)  NOT NULL,
    name varchar(300)  NOT NULL,
    class varchar(300)  NOT NULL,
    FOREIGN KEY (drug_id) REFERENCES Prescriptions(drug)
  );'''
cursor.execute(sql)

###  Loading Cleaned Data into Database

In [ ]:
patients.to_sql('Patients', conn, if_exists='replace', index = False)
conn.commit()

admissions.to_sql('Admissions', conn, if_exists='replace', index = False)
conn.commit()

d_icd_diagnoses.to_sql('D_icd_diagnoses', conn, if_exists='replace', index = False)
conn.commit()

diagnoses_icd.to_sql('Diagnoses_icd', conn, if_exists='replace', index = False)
conn.commit()

prescriptions.to_sql('Prescriptions', conn, if_exists='replace', index = False)
conn.commit()

drug_data.to_sql('Drugs', conn, if_exists='replace', index = False)
conn.commit()